In [ ]:
from loadData import load_data, TRAIN_DATA_POINTS, TEST_DATA_POINTS, N_CLASSES
from processing import generate_class_matrix, generate_test_class_matrix

train_data = load_data("data/ae.train", num_data_points=TRAIN_DATA_POINTS)
test_data = load_data("data/ae.test", num_data_points=TEST_DATA_POINTS)

train_class_matrix = generate_class_matrix(TRAIN_DATA_POINTS, N_CLASSES)
test_class_matrix = generate_test_class_matrix(TEST_DATA_POINTS, N_CLASSES)

The following block of code is run to perform a hyperparameter search on the number of nearest neighbors.

In [ ]:
from knn_pipeline import knn_hyperparameter_search

k_to_search = range(1, 11)
path = "search_results.txt"

# leave-one-out cross validation
knn_hyperparameter_search(
    data=train_data, 
    labels=train_class_matrix, 
    k_to_search=k_to_search, 
    k_folds=TRAIN_DATA_POINTS, 
    results_location=path)

The results of the hyperparameter search for 1 through 10 nearest neighbors are shown by the function in the next block. 
The search is done for two different local distance measures in the Dynamic Time Warping (DTW) procedure. The local distance measures are the absolute distance and the squared distance.

In [ ]:
from visualizeData import show_search_results

show_search_results()

In [ ]:
from knn_pipeline import knn_accuracy

k_nn = 3
accuracy = knn_accuracy(k_nn, test_data, test_class_matrix, train_data, train_class_matrix)
accuracy

In [ ]:
mismatch = 100 - accuracy
mismatch